In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from hive_prd import get_data
from datetime import date
from tqdm import tqdm
import re

In [9]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [45]:
#df = get_data('select * from sp_customer.lp_bv_chat_transcript where time>=\'2017-03-01 00:00:00.0000000000\'')
query = '''select * from customer.dl_cr_customer_canvas 
where contract_start_date>=\'2015-01-01\' and contract_start_date<\'2015-02-01\''''
df = get_data(query)

In [204]:
from tqdm import tqdm

In [388]:
from bs4 import BeautifulSoup
import requests

street_nos = df.sap_supply_point_house_num1.values
addrs = df.supply_address + ' ' +df.sap_supply_point_city+ ' '+ df['sap_supply_point_state']

street_nos = street_nos[100:150]
addrs = addrs[100:150].values

def find_bed_on_realestate(links,street_no):
    bed = 0
    bath = 0
    for link in links:
        r = requests.get(link)
        if street_no is not None and street_no in r.text:
            soup = BeautifulSoup(r.text) #realestatelink
            match = re.search('features":{"general":{"bedrooms":(\d),"bathrooms":(\d)',soup.text)
            if match:
                bed = int(match.group(1))
                bath = int(match.group(2))
                return bed,bath
            match = soup.findAll("span", { "class" : "config-num" })
            if match:
                try:
                    bed = int(match[0].string)
                    bath = int(match[0].string)
                except:
                    continue
                return bed,bath
    return bed,bath 

def find_bed_on_ksou(links,street_no):
    bed = 0
    bath = 0
    for link in links:
        r = requests.get(link)
        if street_no is not None and street_no in r.text:
            soup = BeautifulSoup(r.text) #realestatelink
            match = re.search('(\d) <b>Bed</b> rooms (\d)',str(a))
            if match:
                bed = int(match.group(1))
                bath = int(match.group(2))
                return bed,bath
    return bed,bath     

beds = []
baths = []
for i in tqdm(range(len(addrs)-1),ascii=True):
    r = requests.get('https://www.google.com.au/search?q='+addrs[i].replace(' ','+')+'+bedroom')
    soup = BeautifulSoup(r.text)
    realestatelink = []

    for ti in soup.find_all('h3'):
        for ele in ti.find_all('a'):
            link = ele.get('href')
            if 'realestate.com.au' in link:
                match = re.search(r'/url\?q=(.+)&sa=', link)
                if match is not None:
                    realestatelink.append(match.group(1))

    bed,bath = find_bed_on_realestate(realestatelink,street_nos[i])
    
    
    if bed == 0:
        r = requests.get('https://www.google.com.au/search?q='+addrs[i].replace(' ','+')+'+ksou+bedroom')
        soup = BeautifulSoup(r.text)
        
        desc = soup.findAll("span", { "class" : "st" })
        for item in desc:
            match = re.search('(\d) <b>Bed</b> rooms (\d)',str(item))
            if match:
                bed = int(match.group(1))
                bath = int(match.group(2))
    
    beds.append(bed)
    baths.append(bath)

  0%|                                                   | 0/49 [00:00<?, ?it/s]C:\Users\A138262\AppData\Local\Continuum\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
100%|##########################################| 49/49 [02:34<00:00,  3.18s/it]
